In [ ]:
!pip install -Uq langchain langchain-community

In [ ]:
!pip install -Uq unstructured unstructured[pdf]

In [ ]:
!pip install -Uq sentence-transformers

In [ ]:
!pip install -Uq langchain-pinecone pinecone-client chromadb langchain-chroma

In [ ]:
!pip install -Uq langchain-groq

In [ ]:
import os
from google.colab import userdata

In [ ]:
from langchain_community.document_loaders import UnstructuredFileLoader
from unstructured.cleaners.core import clean_extra_whitespace
from langchain.embeddings import HuggingFaceEmbeddings

# from pinecone import Pinecone, ServerlessSpec
# from langchain_pinecone import PineconeVectorStore
# import pinecone

In [ ]:
# from unstructured.partition.pdf import PDFPageLoader
# from unstructured.cleaners.core import clean_extra_whitespace


In [ ]:
folder_name = 'user_documents'
if not os.path.exists(folder_name):
  os.makedirs(folder_name)
  print("Directory created...")
else:
  print("Directory already exist...")

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
for file_name in uploaded:
  os.rename(file_name, f'{folder_name}/{file_name}')
print("Uploaded files moved to user_documents")

In [ ]:
print("List of files uploaded are: ")
dir = os.listdir(folder_name)
for i in dir :
  if i == '.ipynb_checkpoints':
    dir.pop()
    print("popped: ", i)
    continue
  print(" - ", i)

In [ ]:
%pip install --upgrade --quiet unstructured

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [ ]:
for file in dir:
  print(file)
  loader = UnstructuredPDFLoader(
      f'user_documents/{file}',
      mode='elements',
      strategy='fast',
      postprocessors=[clean_extra_whitespace]
  )
doc = loader.load()

In [ ]:
print(doc[1].metadata)
print(doc[1].page_content)

In [ ]:
doc_chunk = []
for count in range(1, doc[-1].metadata['page_number']):
  string = ""
  print("Page: ", count)
  for i in doc:
    if i.metadata['page_number'] == count:
      string += " " + i.page_content
  print(string)
doc_chunk.append(string)

In [ ]:
# for file in dir :
#   print(file)
#   loaded = UnstructuredFileLoader(
#       f'user_documents/{file}',
#       mode='elements',
#       strategy='fast',
#       post_processors=[clean_extra_whitespace]
#   )
#   doc = loaded.load()

In [ ]:
# doc_chunk = []
# for count in range(1, doc[-1].metadata['page_number']):
#   string = ""
#   print("Page: ", count)
#   for i in doc:
#     if i.metadata['page_number'] == count:
#       string += i.page_content
#   doc_chunk.append(string)
#   print(string)

In [ ]:
# # Load the document by pages
# loaded = PDFPageLoader(
#     f'user_documents/ubuntu-server-guide-2022-08-03.pdf',
#     mode='elements',
#     strategy='fast',
#     post_processors=[clean_extra_whitespace]
# )

# # Iterate over pages and load them
# docs_by_page = []
# for i, page in enumerate(loaded.load()):
#     page_doc = loaded.load(page_number=i + 1)  # Page numbers are 1-indexed
#     docs_by_page.append(page_doc)


In [ ]:
# docs_by_page = []
# for i, page in enumerate(doc, 1):
#     page_doc = loaded.load(page_number=i)  # Page numbers are 1-indexed
#     docs_by_page.append(page_doc)


In [ ]:
# doc

In [ ]:
# doc_chunk = []
# for count in range(1, doc[-1].metadata['page_number']):
#   string = ""
#   print("Page: ", count)
#   for i in doc:
#     if i.metadata['page_number'] == count:
#       string += i.page_content
#   doc_chunk.append(string)
#   print(string)

In [ ]:
# doc_chunk

In [ ]:
embedding_model_1 = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
embedding_model_2 = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
if not os.path.exists('chroma_db'):
  os.makedirs('chroma_db')
  print("Directory created...")
else:
  print("Directory already exist...")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embedding_model_1,
    persist_directory="./chroma_db"
)

In [ ]:
from uuid import uuid4

In [ ]:
from langchain_core.documents import Document

In [ ]:
doc_chunk = []
for count in range(1, doc[-1].metadata['page_number']):
  string = ""
  for i in doc:
    if i.metadata['page_number'] == count:
      string +=i.page_content
  document = Document(
      page_content = string,
      metadata = {
          "filename": doc[0].metadata["filename"],
          "filetype": doc[0].metadata["filetype"],
          "page_number": i.metadata["page_number"]
      }
  )
  doc_chunk.append(document)

In [ ]:
doc[0].metadata

In [ ]:
doc_chunk[0]

In [ ]:
for doc in doc_chunk:
  vector_store.add_documents(documents=doc.page, ids=str(uuid4()))

In [ ]:
document_1 = Document(
    page_content="Ubuntu Server GuideWelcome to the Ubuntu Server Guide! This site includes information on using Ubuntu Server for Ubuntu 20.04 LTS (Focal Fossa) and later . For an offline version as well as versions for previous releases see below.Getting Started:Basic Install Automated Install Cloud Images Software Containers Run Ubuntu Server software in dedicated containersInstall Server Edition using the installer Automate installation using autoinstall Run Ubuntu Server Edition on a public cloud providerRefining Your Infrastructure:Networking Networked Storage Configure multiple I/O paths for storage Security Virtualization High Availability MonitoringRegister network devicesProtect your server and network Separate your services into virtualized environments Cluster systems redundantly to minimize down-time Ensure infrastructure availabilityConfiguring Your Services:Databases Backups Mail Servers Web ServersInstall database servers Establish your backup plans Set up e-mail services Set up web servicesSupportThere are a couple of different ways that the Ubuntu Server edition is supported: commercial support and community support. The main commercial support (and development funding) is available from Canonical, Ltd. They supply reasonably- priced support contracts on a per desktop or per-server basis. For more information see the Ubuntu Advantage page.Community support is also provided by dedicated individuals and companies that wish to make Ubuntu the best distribution possible. Support is provided through multiple mailing lists, IRC channels, forums, blogs, wikis, etc. The large amount of information available can be overwhelming, but a good search engine query can usually provide an answer to your questions. See the Ubuntu Support page for more information.Improving the DocumentationIf you find any errors or have suggestions for improvements to pages, please use the link at the bottom of each topic titled: “Help improve this document in the forum.” This link will take you to the Server Discourse forum for the specific page you are viewing. There you can share your comments or let us know about bugs with any page.PDFs and Previous ReleasesBelow are links to the previous Ubuntu Server release server guides as well as an offline copy of the current version of this site:Ubuntu 20.04 LTS (Focal Fossa) and later: PDF Ubuntu 18.04 LTS (Bionic Beaver): Web and PDFNavigationNavigationLevel PathNavlink1 0Introduction Getting started1",
    metadata={'filename': 'ubuntu-server-guide-2022-08-03.pdf', 'filetype': 'application/pdf', 'page_number': 383},
    id=1,
)
vector_store.add_documents(documents=[document_1], ids=[str(uuid4())])
